## Initialize environment

Note: Before executing this notebook on colab, go to the secrets on the left panel 🔐, 
click on Gemini API keys, import key from Google AI studio. 


In [1]:
! uv pip install agentics-py

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    from google.colab import userdata
    os.environ['GEMINI_API_KEY'] = userdata.get('GOOGLE_API_KEY')
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.11 environment at: /Users/nahueldefosse/workspace/nl2insights/agentics/.venv
Audited 1 package in 152ms
In Colab: False


## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator. Similarly you can turn any function of your choice into a tool. 

In [2]:
from ddgs import DDGS
from crewai.tools import tool
## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("fetch_data_async")
def web_search(query: str) -> str:
    """ Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))    


## Define your agent, task and crew

Note that the agent is provided with memory = True This provides the agent with conversation history. By activating memory=True in the crew, we make sure that the agents persists across multiple crew session calls, with different input states. 

In [ ]:
from crewai import Agent, Task, Crew, Process
from agentics.core.llm_connections import get_llm_provider
# Create a conversational agent with a friendly role/goal
chat_agent = Agent(
    llm=get_llm_provider(provider_name='gemini'),
    role="Helpful Assistant",
    goal="Have a natural multi-turn conversation",
    backstory="You are a friendly assistant that remembers context and asks for clarification when needed.",
    memory=True  # enable memory for conversational context. 
)

# Define a simple Task that represents a single AI response
task = Task(
    description="Respond appropriately to user's message, maintaining context. {input}",
    expected_output="A conversational reply",
    agent=chat_agent,
    tools=[web_search]
)

crew = Crew(
    agents=[chat_agent],
    tasks=[task],
    memory=True,  # ensure conversation state persists across turns
)

2025-09-10 15:29:43.087 | DEBUG    | agentics.core.llm_connections:<module>:121 - AGENTICS is connecting to the following LLM API providers:
2025-09-10 15:29:43.087 | DEBUG    | agentics.core.llm_connections:<module>:124 - 0 - WatsonX
2025-09-10 15:29:43.087 | DEBUG    | agentics.core.llm_connections:<module>:129 - 1 - Gemini
2025-09-10 15:29:43.088 | DEBUG    | agentics.core.llm_connections:<module>:135 - Please add API keys in .env file to add or disconnect providers.
2025-09-10 15:29:43.101 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-10 15:29:43.102 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini']. Using 'watsonx'
2025-09-10 15:29:43.104 | DEBUG    | agentics.core.llm_connections:get_llm_provider:42 - Using specified LLM provider: watsonx


## Run the crew in a conversation loop
Since memory has been set to True, the crew will get access to the previous round of conversations, exibiting a conversational behaviour

In [4]:
while user_input := input("You: ").strip():
    print("\n\n===========\nYou: ",user_input)
    result = crew.kickoff(inputs={"input": user_input})
    print("\nAssistant:", result)



You:  what's the warlus operator


ERROR:root:Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'



Assistant: The walrus operator in Python, denoted by :=, is an assignment expression that allows you to assign a value to a variable within an expression. It's called the walrus operator because its syntax resembles the eyes and tusks of a walrus. This operator can simplify your code by combining assignment and use in a single step, making it useful when you need to use a value multiple times in a loop. It was introduced in Python 3.8 and is also known as an assignment expression. The walrus operator can be used in various contexts, such as in if statements, while loops, and list comprehensions, to make your code more concise and readable. For example, you can use it to assign a value to a variable and then use that variable in the same expression, like this: if (n := len(my_list)) > 5: print(f"List has {n} elements"). This way, you don't need to assign the value to a variable in a separate line before using it. The walrus operator is a powerful tool in Python that can help you write 

ERROR:root:Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'



Assistant: Yes, you can use a variable in a for loop. In programming, a for loop is used to iterate over a sequence (such as a list, tuple, dictionary, set, or string) or other iterable objects. The variable you use in the for loop is often called the loop variable, and it takes the value of the next item in the sequence on each iteration of the loop. For example, if you have a list of numbers and you want to print each number, you can use a for loop with a variable to represent each number in the list. 

Here is a simple example in Python:
```
numbers = [1, 2, 3, 4, 5]
for number in numbers:
    print(number)
```
In this example, the variable "number" is used to represent each item in the list "numbers" on each iteration of the loop. This will output:
```
1
2
3
4
5
```
You can use any valid variable name in place of "number" to represent the items in the sequence you are iterating over. 

It's worth noting that the specific syntax and capabilities of for loops can vary depending on t

KeyboardInterrupt: Interrupted by user